
## **pixel-level-segmentation-in-pets-unipr**

@author Matteo Gianvenuti https://github.com/Mqtth3w 
@license GPL-3.0


# Find the best model

### Contents

- Fixed hyperparameters and general information (UNet and DeepLabV3)
- Find the best optimizer and loss (UNet and DeepLabV3)
- Find the best dropout probability and batch size (UNet and DeepLabV3)
- Find the best learning rate (UNet)
- Find the best weight decay (UNeT)
- Find the best learning rate (DeepLabV3)
- Find the best weight decay (DeepLabV3)


### Fixed hyperparameters and general information (UNet and DeepLabV3)

- **epochs**: As maximun number of epochs I used 50 to train the model as musch as possible without exaggerating. Then to see how the training and test loss curves behave, surprisinly the best model configurations still had room of improvement also at 50 epochs. The average curves trend was bearish
- **patience**: I used always six as patience for early stopping, this to have a balcend value, not too high but also not too little
- **patience2**: This value tells the scheduler how many steps without improvement to wait before changing the learning rate to maximize the IoU. It should be lower than the previous, I used 5 to give the model some time to train
- **momentum**: I used the standard choice 0.9

I alwasy used **normalized data** because it leads to better results. Initially, I used a custom normalization with mean and std calculated on the dataset but the images were compressed in a very little range and it did not work so I passed to the mean and std for RGB images calucalted on ImagedNet.

To improve the training and dfferentiate the data along the epochs I always used the **agumentations**: a random horizontal flip and a random color jitter added after the optimizer and loss function choices.

### Find the best optimizer and loss (UNet and DeepLabV3)

I tested Adam because it is robust to the initial learning rate choice and because it is considered the state of the art. I also wanted to test RSMprop because a note implementation of UNet in PyTorch with good results use it. 
As loss function I tested:
- The dice loss because it focuses on the overlap between the predicted and the ground truth mask. So I expect it to have the best IoU 
- The Cross-entropy loss because the elements in the mask can belong to three classes: the background, the pet edge and the pet
- Their combination (loss1 + loss2)

<br>

| Model/run name | Optimizer | loss | IoU | L1 distance | 
|-------------|-------------|-------------|-------------|-------------|
| **UNet_Adam_dice** | **Adam** | **Dice** | **0.8165** | **0.0704** |
| UNet_Adam_CE | Adam | Cross-entropy | 0.8116 | 0.0722 |
| UNet_Adam_combo | Adam | Dice + Cross-entropy | 0.8063 | 0.0748 |
| UNet_RSMprop_dice | RSMprop | Dice | 0.8152 | 0.0709 |
| UNet_RSMprop_CE | RSMprop | Cross-entropy | 0.8041 | 0.0753 |
| UNet_RSMprop_combo | RSMprop | Dice + Cross-entropy | 0.7809 | 0.0857 |

<br>

As expected Adam optimizer and Dice loss are the best for this task. Then I used this configuration for both UNet and DeepLabV3.

### Find the best dropout probability and batch size (UNet and DeepLabV3)

After each hyperparameters research I used the one with highest score as default for the successive run. 
For the following test I also used 1e-6 as weight decay.

<br>

| Model/run name | Dropout p | Batch size | IoU | L1 distance | 
|-----------|-----------|-----------|-----------|-----------|
| UNet_dr015_bs32_lr3e_4 | 0.15 | 32 | 0.8102 | 0.0731 |
| **UNet_dr01_bs32_lr3e_4** | **0.1** | **32** | **0.8377** | **0.0615** |
| UNet_dr015_bs16_lr2e_4 | 0.15 | 16 | 0.8376 | 0.0615 |
| UNet_dr01_bs16_lr2e_4 | 0.1 | 16 | 0.8295 | 0.0648 |
| UNet_dr015_bs8_lr1e_4 | 0.15 | 8 | 0.8181 | 0.0697 |
| UNet_dr01_bs8_lr1e_4 | 0.1 | 8 | 0.8270 | 0.0659 |

<br>

As result the best dropout probability is 10% with 32 as batch size. For DeepLabV3 dropout is already defined and integrated inside the model structure.
A higher batch size has less noisy gradient and hence more stable results.

### Find the best learning rate (UNet)

<br>

| Model/run name | Learning rate | IoU | L1 distance | 
|-----------|-----------|-----------|-----------|
| UNet_lr1e_3 | 1e-3 | 0.8368 | 0.0618 |
| UNet_lr2e_3 | 2e-3 | 0.6824 | 0.1322 |
| UNet_lr5e_4 | 5e-4 | 0.8226 | 0.0679 |
| UNet_lr7e_5 | 7e-5 | 0.8293 | 0.0649 |

<br>

The best configuration here did not outperform the one found previously with the learning rate 3e-4.

### Find the best weight decay (UNet)

<br>

| Model/run name | Weight decay | IoU | L1 distance | 
|-----------|-----------|-----------|-----------|
| UNet_wd_1e_5 | 1e-5 | 0.8279 | 0.0655 |
| **UNet_wd_1e_4** | **1e-4** | **0.8473** | **0.0574** |

<br>

Overall, the best UNet model is UNet_wd_1e_4.

### Find the best learning rate (DeepLabV3)

For the following test I used 1e-6 as weight decay.
<br>

| Model/run name | Learning rate | IoU | L1 distance | 
|-----------|-----------|-----------|-----------|
| DeepLabV3ResNet_lr2e_4 | 2e-4 | 0.8911 | 0.0396 |
| DeepLabV3ResNet_lr3e_4 | 3e-4 | 0.8910 | 0.0397 |
| DeepLabV3ResNet_lr4e_4 | 4e-4 | 0.8926 | 0.0391 |
| **DeepLabV3ResNet_lr5e_4** | **5e-4** | **0.8935** | **0.0387** |
| DeepLabV3ResNet_lr7e_5 | 7e-5 | 0.8868 | 0.0412 | 

<br>
So the best learning rate found for this net is 5e-4.

### Find the best weight decay (DeepLabV3)

<br>

| Model/run name | Weight decay | IoU | L1 distance | 
|-----------|-----------|-----------|-----------|
| DeepLabV3ResNet_wd1e_5 | 1e-5 | | |
| DeepLabV3ResNet_wd1e_4 | 1e-4 | | |

<br>


In [3]:
# for more details about the runs you can see the charts with tensorboard # http://localhost:6006

#import os
#print(os.getcwd())

%load_ext tensorboard
#%reload_ext tensorboard

%tensorboard --logdir=../unet/runs 
%tensorboard --logdir=../deeplabv3_resnet101/runs

Reusing TensorBoard on port 6007 (pid 1996), started 0:00:22 ago. (Use '!kill 1996' to kill it.)

Reusing TensorBoard on port 6008 (pid 13964), started 0:00:11 ago. (Use '!kill 13964' to kill it.)